<a href="https://colab.research.google.com/github/Zhaozhufeng1/Zhaozhufeng1-Document-Retrieval-and-Question-Answering-with-LLMs/blob/main/Copy_of_Generative_AI_Document_Retrieval_and_Question_Answering_with%C2%A0LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI - Document Retrieval and Question Answering with LLMs


## Authenticate

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:

!gcloud config set project sascha-playground-doit

Are you sure you wish to set property [core/project] to sascha-playground-doit?

Do you want to continue (Y/n)?  

Command killed by keyboard interrupt

^C


## Dependencies

In [ ]:
!pip install google-cloud-aiplatform==1.25.0
!pip install langchain==0.0.187
!pip install xmltodict==0.13.0
!pip install unstructured==0.7.0 # used by langchain
!pip install pdf2image==1.16.3 #used by langchain
!pip install requests==2.31.0
!pip install beautifulsoup4==4.12.2

In [ ]:
PROJECT_ID = 'sascha-playground-doit'
REGION = 'us-central1'
BUCKET = 'gs://doit-llm/embeddings'
DIMENSIONS=768
DISPLAY_NAME='palm-langchain-document-answering'
ENDPOINT='us-central1-aiplatform.googleapis.com'
TEXT_GENERATION_MODEL='text-bison@001'

sitemap='https://cloud.google.com/vertex-ai/sitemap.xml'

In [ ]:
import os
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

# Documents
## Parse the sitemap

In [ ]:
import requests
from bs4 import BeautifulSoup

def parse_sitemap(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "xml")
    urls = [element.text for element in soup.find_all("loc")]
    return urls

sites = parse_sitemap(sitemap)

In [ ]:
sites_filtered = [url for url in sites if '/reference/' not in url and '?hl' not in url]
sites_filtered

In [ ]:
len(sites_filtered)

## Load page content using LangChains UnstructuredURLLoader

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=sites_filtered)
documents = loader.load()

In [ ]:
documents[0]

In [ ]:
len(documents)

## Chunking

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 100)

document_chunks = text_splitter.split_documents(documents)

print(f"Number documents {len(documents)}")
print(f"Number chunks {len(document_chunks)}")

document_chunks=[f"Context: {chunk.page_content} Source: {chunk.metadata['source']}" for chunk in document_chunks]

# Embeddings for documents



## Create embedding for all document chunks

In [ ]:
!rm -rf ./documents
!mkdir ./documents

In [ ]:
import time
from langchain.embeddings import VertexAIEmbeddings
import pandas as pd
import json

def handle_quota_errors(func, *args, retry_delay=5,backoff_factor=2, **kwargs):
    retries = 0

    try:
      return func(*args, **kwargs)
    except Exception as e:
      print(f"error: {e}")
      retries += 1
      wait = retry_delay * (backoff_factor ** retries)
      time.sleep(wait)
      print("wait for {wait} seconds")


embeddings = VertexAIEmbeddings()

df = pd.DataFrame(document_chunks, columns =['text'])

index_embeddings = []

for index, doc in df.iterrows():
  print(f"Get embedding and write document for document {index} of {len(df)-1}")
  embedding = handle_quota_errors(embeddings.embed_query, doc['text'])

  if embedding is not None:

    doc_id=f"{index}.txt"
    embedding_dict = {
              "id": doc_id,
              "embedding": [str(value) for value in embedding],
    }
    index_embeddings.append(json.dumps(embedding_dict) + "\n")

    doc_id = f"{index}.txt"
    with open(f"documents/{doc_id}", "w") as document:
      document.write(doc['text'])


with open("embeddings.json", "w") as f:
    f.writelines(index_embeddings)

## Copy document chunks and embeddings to Google Cloud Storage for later indexing and retreival steps

In [ ]:
!gsutil cp -r documents  gs://doit-llm/documents
!gsutil cp embeddings.json gs://doit-llm/embeddings/embeddings.json

# Index

## Option 1 - Create Streaming Index

In [ ]:
%%writefile index.json
{
    displayName: "palm-langchain-document-answering",
    description: "palm-langchain-document-answering",
    metadata: {
       contentsDeltaUri: "gs://doit-llm/embeddings",
       config: {
          dimensions: "768",
          approximateNeighborsCount: 150,
          distanceMeasureType: "DOT_PRODUCT_DISTANCE",
          algorithmConfig: {
             bruteForceConfig: {}
          }
       },
    },
    indexUpdateMethod: "STREAM_UPDATE"
}

In [ ]:
!curl -X POST -H "Content-Type: application/json" \
-H "Authorization: Bearer `gcloud auth print-access-token`" \
https://us-central1-aiplatform.googleapis.com/v1/projects/sascha-playground-doit/locations/us-central1/indexes \
-d @index.json

## Option 2 - Create Batch Index

In [ ]:
index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
      display_name=DISPLAY_NAME,
      contents_delta_uri=BUCKET,
      dimensions=DIMENSIONS,
      approximate_neighbors_count=150,
      distance_measure_type="DOT_PRODUCT_DISTANCE")

## Create Endpoint

In [ ]:
REGION = "us-central1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

PROJECT_ID = "sascha-playground-doit"
PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

NETWORK_NAME = "matching-engine-vpc-network"

VPC_NETWORK_NAME = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, NETWORK_NAME)
VPC_NETWORK_NAME

In [ ]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    network=VPC_NETWORK_NAME,
    #IMPORTANT if you want to use a public endpoint you need to use aiplatform_v1beta1 when query or inserting vectors
    # https://cloud.google.com/vertex-ai/docs/matching-engine/deploy-index-public
    # public_endpoint_enabled=True
)

## Deploy Index

In [ ]:
# uncomment if you want to use existing index or endpoint

INDEX_RESOURCE_NAME = "projects/234439745674/locations/us-central1/indexes/8624771518414454784"
index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

ENDPOINT_RESOURCE_NAME = "projects/234439745674/locations/us-central1/indexEndpoints/1017628797832265728"
index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=ENDPOINT_RESOURCE_NAME)

In [ ]:
deployed_index = index_endpoint.deploy_index(
    index=index, deployed_index_id=DISPLAY_NAME.replace('-','_')
)

# The fun part (needs to be inside the same VPC)
Matching Engine now supports in beta public endpoints. Stayed tuned for a dedicated article on that.


In [ ]:
INDEX='8624771518414454784'
ENDPOINT='1017628797832265728'
DOCS_BUCKET='doit-llm'

from langchain.embeddings import VertexAIEmbeddings
embeddings = VertexAIEmbeddings()

In [ ]:
from langchain.vectorstores.matching_engine import MatchingEngine
from langchain.agents import Tool

def matching_engine_search(question):

    vector_store = MatchingEngine.from_components(
                        index_id=INDEX,
                        region=REGION,
                        embedding=embeddings,
                        project_id=PROJECT_ID,
                        endpoint_id=ENDPOINT,
                        gcs_bucket_name=DOCS_BUCKET)

    relevant_documentation=vector_store.similarity_search(question, k=8)
    context = "\n".join([doc.page_content for doc in relevant_documentation])[:10000]
    return str(context)

In [ ]:
question = "what is the latency for Vertex AI Matching Engine?"

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

matching_engine_response=matching_engine_search(question)

prompt=f"""
Follow exactly those 3 steps:
1. Read the context below and aggregrate this data
Context : {matching_engine_response}
2. Answer the question using only this context
3. Show the source for your answers
User Question: {question}


If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
"""

model = TextGenerationModel.from_pretrained(TEXT_GENERATION_MODEL)
response = model.predict(
        prompt,
        temperature=0.2,
        top_k=40,
        top_p=.8,
        max_output_tokens=1024,
)

print(f"Question: \n{question}")
print(f"Response: \n{response.text}")

## Direct usage of Vertex AI Matching Engine (only to showase the direct usage for document retrieval)

In [ ]:
embedding = embeddings.embed_query("what is the latency for Vertex AI Matching Engine?")
print(embedding)
response = index_endpoint.match(
    deployed_index_id="palm-langchain-document-answering",
    queries=[embedding]
)

response

# Little Index Helper

In [ ]:
!gcloud ai index-endpoints list --project="sascha-playground-doit" --region="us-central1"

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
createTime: '2023-06-09T15:11:55.283483Z'
deployedIndexes:
- automaticResources:
    maxReplicaCount: 2
    minReplicaCount: 2
  createTime: '2023-06-09T15:12:02.692696Z'
  deploymentGroup: default
  id: catawiki_langchain_document_answering
  index: projects/234439745674/locations/us-central1/indexes/1910889636423532544
  indexSyncTime: '2023-06-09T19:06:23.127411Z'
  privateEndpoints:
    matchGrpcAddress: 10.78.0.5
description: catawiki-langchain-document-answering
displayName: catawiki-langchain-document-answering
etag: AMEw9yNOk1n2jUEW4GaC_Ge3Oin5gvEr7OGhQpjJMkrVdV3ReJdf6MqMX5MB-1LWoYU=
name: projects/234439745674/locations/us-central1/indexEndpoints/4187459248059318272
network: projects/234439745674/global/networks/matching-engine-vpc-network
updateTime: '2023-06-09T15:11:56.042078Z'
---
createTime: '2023-06-08T18:12:58.604542Z'
deployedIndexes:
- automaticResources:
    maxReplicaCount: 2
    minReplicaCount: 2


In [ ]:
!gcloud ai index-endpoints undeploy-index "4187459248059318272" \
  --deployed-index-id="catawiki_langchain_document_answering" \
  --project="sascha-playground-doit" \
  --region="us-central1"

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
'@type': type.googleapis.com/google.cloud.aiplatform.v1.UndeployIndexResponse


In [ ]:
!gcloud ai indexes list \
  --project="sascha-playground-doit" \
  --region="us-central1"

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
createTime: '2023-06-09T14:00:46.083298Z'
displayName: catawiki-langchain-document-answering
etag: AMEw9yOo6dzj-x-ZvMTxRgC6s7hLLRtHs35JluhdeVYbIEENGEKqv1d4iEVEKytf1U3B
indexStats:
  shardsCount: 1
  vectorsCount: '2107'
indexUpdateMethod: BATCH_UPDATE
metadata:
  config:
    algorithmConfig:
      treeAhConfig: {}
    approximateNeighborsCount: 150
    dimensions: 768
    distanceMeasureType: DOT_PRODUCT_DISTANCE
    shardSize: SHARD_SIZE_MEDIUM
metadataSchemaUri: gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml
name: projects/234439745674/locations/us-central1/indexes/1910889636423532544
updateTime: '2023-06-09T15:04:45.339416Z'
---
createTime: '2023-06-09T09:34:02.531861Z'
deployedIndexes:
- deployedIndexId: image_similarity_vpc
  indexEndpoint: projects/234439745674/locations/us-central1/indexEndpoints/2199120012575244288
description: image-similarity
displayName: image-

In [ ]:
!gcloud -q ai indexes delete "projects/234439745674/locations/us-central1/indexes/1910889636423532544" \
  --project="sascha-playground-doit" \
  --region="us-central1"

Using endpoint [https://us-central1-aiplatform.googleapis.com/]


In [ ]:
!gcloud ai index-endpoints delete 5731859668712554496 \
  --project="sascha-playground-doit" \
  --region="us-central1" \
  --quiet